In [1]:
import pandas as pd
import geopandas as gpd
import shapely.geometry as geom
import numpy as np
import json

In [2]:

gdf : gpd.GeoDataFrame = gpd.read_file('local/MB_2021_AUST_SHP_GDA2020/MB_2021_AUST_GDA2020.shp')
# 4m

In [3]:
gdf : gpd.GeoDataFrame = gdf[['MB_CODE21', 'geometry']].rename(columns={'MB_CODE21': 'id'})

In [4]:
gdf_geom : gpd.GeoSeries = gdf.boundary
# 30s

In [5]:
# Combine multiple linestring and multilinestring geometries into single multilinestring geometry
gdml = gdf_geom.apply(lambda x: geom.MultiLineString([x]) if isinstance(x, geom.LineString) else x)
# 1m
# 30s

In [6]:
gdl = gdml.apply(lambda x: x.geoms if x is not None else None).explode()
# 1m

In [7]:
gdl.dropna(inplace=True)

In [8]:
sf_lines = gdl.apply(lambda x: np.array(x.coords))
# 1m

In [9]:
# Get all points
points_full = np.concatenate(sf_lines.to_numpy())
# points_full.sort(axis=0)
len(points_full)
# 19647448
# 10s

19647448

In [20]:
sf_lines_len = sf_lines.transform(lambda x: np.array(range(len(x))))
# 10s

In [23]:
points_order = sf_lines_len.explode().reset_index().rename(columns={'index': 'line', 0: 'order'})
# 20s

In [27]:
points_order = points_order.to_numpy()
# 20s

In [28]:
points_order

array([[0, 0],
       [0, 1],
       [0, 2],
       ...,
       [368283, 2],
       [368283, 3],
       [368283, 4]], dtype=object)

In [30]:
points = np.concatenate([points_order, points_full], axis=1)

In [32]:
df = pd.DataFrame(points, columns=['line', 'order', 'lon', 'lat'])

In [35]:
dfp = df[['lon', 'lat']].drop_duplicates().reset_index(drop=True)
# 2m

In [37]:
dfp = dfp.reset_index(drop=False).rename(columns={'index': 'point_id'})
# 5s

In [40]:
df_lines = pd.merge(df, dfp, on=['lon', 'lat'], how='left')
# 5m

In [42]:
df_lines.drop(columns=['lon', 'lat'], inplace=True)
# 2m

In [44]:
df_lines.to_csv('local/mb_lines.csv', index=False)
# 2m

In [45]:
dfp.to_csv('local/mb_points.csv', index=False)
# 1m 30s

In [53]:
gdf.dropna()

,id,geometry
0,10000010000,"POLYGON ((146.92785 -36.08365, 146.92815 -36.0..."
1,10000021000,"POLYGON ((146.93543 -36.04752, 146.93722 -36.0..."
2,10000022000,"POLYGON ((146.93040 -36.04996, 146.93054 -36.0..."
3,10000023000,"POLYGON ((146.93254 -36.05238, 146.93101 -36.0..."
4,10000024000,"POLYGON ((146.93044 -36.04733, 146.93036 -36.0..."
...,...,...
368279,90000733000,"POLYGON ((167.99423 -29.04525, 167.99414 -29.0..."
368280,97500000000,"POLYGON ((150.69854 -35.13603, 150.69079 -35.1..."
368281,97000001777,"POLYGON ((105.66649 -10.42203, 105.66649 -10.4..."
368282,97000002777,"POLYGON ((96.88760 -12.11086, 96.88852 -12.110..."


In [72]:
sf_poly_lines : np.ndarray = gdml.dropna().apply(lambda x: np.array(range(len(x.geoms)))).to_numpy()
# 10s

In [65]:
arr_block_id : np.ndarray = gdf.dropna()['id'].to_numpy()

In [82]:
df_pl = pd.DataFrame({'block_id': arr_block_id, 'lines': sf_poly_lines}).explode('lines')

In [89]:
df_pl['points'] = sf_lines_len.reset_index(drop=True, inplace=False)

In [93]:
df_pl = df_pl.explode('points')
# 2m

In [96]:
df_pl

,block_id,lines,points
0,10000010000,0,0
0,10000010000,0,1
0,10000010000,0,2
0,10000010000,0,3
0,10000010000,0,4
...,...,...,...
368254,97000003777,0,34
368254,97000003777,0,35
368254,97000003777,0,36
368254,97000003777,0,37


In [97]:
df

,line,order,lon,lat
0,0,0,146.927851,-36.083648
1,0,1,146.928145,-36.082104
2,0,2,146.92834,-36.081122
3,0,3,146.929139,-36.081218
4,0,4,146.929059,-36.081643
...,...,...,...,...
19647443,368283,0,167.972812,-29.016558
19647444,368283,1,167.972812,-29.015658
19647445,368283,2,167.97384,-29.015658
19647446,368283,3,167.97384,-29.016558
